# Approximation of individual chords
## Ginsel et. al. 2022, Experiment A

In [8]:
%cd ..
from evoaudio.sample_library import SampleLibrary
from evoaudio.base_algorithms import approximate_piece
from evoaudio.population import Population
from evoaudio.mutations import Mutator
from evoaudio.pitch import Pitch
from evoaudio.individual import BaseIndividual
from evoaudio.fitness import fitness

### Approximation of pitches only

In [3]:
POPSIZE = 10
N_OFFSPRING = 1
MAX_STEPS = 1000
ALPHA = 5
BETA = 10
L_BOUND = 1
U_BOUND = 20
ZETA = 0.9954

In [4]:
sample_lib = SampleLibrary()

Loading samples: 100%|██████████| 6826/6826 [00:13<00:00, 496.78it/s]


In [5]:
# Create sample set
target_chords = [
    [("Trumpet", Pitch.c4)],
    [("Violin", Pitch.c4)],
    [("Piano", Pitch.c4)],
    [("Trumpet", Pitch.c4), ("Trombone", Pitch.e4)],
    [("Violin", Pitch.c4), ("Viola", Pitch.e4)],
    [("Piano", Pitch.c4), ("Piano", Pitch.e4)],
    [("Trumpet", Pitch.c4), ("Trombone", Pitch.e4), ("Tuba", Pitch.c3), ("Trumpet", Pitch.g4)],
    [("Violin", Pitch.c4), ("Viola", Pitch.e4), ("Cello", Pitch.c3), ("Violin", Pitch.g4)],
    [("Piano", Pitch.c4), ("Piano", Pitch.e4), ("Piano", Pitch.g4), ("Piano", Pitch.c3)],
    [("Cello", Pitch.c3), ("Trumpet", Pitch.c4), ("Trumpet", Pitch.e4), ("Piano", Pitch.g4), ("Piano", Pitch.c5)]    
]
samples = [[sample_lib.get_sample(instrument=note[0], pitch=note[1]) for note in chord] for chord in target_chords ]
target_individuals = [BaseIndividual() for i in range(len(samples))]
for i, target in enumerate(target_individuals):
    target.samples = samples[i]
target_mixes = [individual.to_mixdown() for individual in target_individuals]

With a-priori knowledge of instruments, approximate only pitches

In [21]:
def get_valid_sample(sample_lib, instrument):
    try:
        return sample_lib.get_sample(instrument=instrument, pitch=sample_lib.get_random_pitch_for_instrument_uniform(instrument, sample_lib.get_random_style_for_instrument(instrument)))
    except:
        return get_valid_sample(sample_lib, instrument)
# Initialize populations with a-priori knowledge
populations = [Population() for _ in range(len(target_chords))]
for i, pop in enumerate(populations):
    for j in range(POPSIZE):
        individual = BaseIndividual()
        for note in target_chords[i]:
            individual.samples.append(get_valid_sample(sample_lib, note[0]))
        individual.fitness = fitness(target_mixes[i], individual.to_mixdown())
        individual.fitness_per_onset.append(individual.fitness)
        pop.insert_individual(individual)
# Only allow the mutate_pitch mutation
mutator = Mutator(sample_library=sample_lib, alpha=ALPHA, beta=BETA, l_bound=L_BOUND, u_bound=U_BOUND, choose_mutation_p=[0, 0, 1]) 

Could not find sample: Trombone, BassTrombone_SessionHornsPro, 76
Could not find sample: Trombone, TenorTrombone_SessionHornsPro, 36
Could not find sample: Violin, tr151VNSOM, 91
Could not find sample: Trombone, TenorTrombone_SessionHornsPro, 33
Could not find sample: Trombone, TenorTrombone_SessionHornsPro, 33
Could not find sample: Trumpet, tr212TRNOM, 89
Could not find sample: Trombone, TenorTrombone_SessionHornsPro, 27
Could not find sample: Trumpet, tr211TRVIM, 84
Could not find sample: Trombone, BassTrombone_SessionHornsPro, 79
Could not find sample: Tuba, Tuba_SessionHornsPro, 67
Could not find sample: Violin, tr152VNNOM, 55
Could not find sample: Piano, tr013PFPEM, 27
Could not find sample: Piano, tr011PFNOM, 22
Could not find sample: Piano, tr012PFPM, 106
Could not find sample: Piano, tr011PFPEM, 91
Could not find sample: Piano, tr013PFPEM, 102
Could not find sample: Piano, tr011PFPEM, 99
Could not find sample: Piano, tr011PFNOM, 93
Could not find sample: Trumpet, tr211TRVIM, 

In [22]:
results = []
for i, pop in enumerate(populations):
    result = approximate_piece(target_y=target_mixes[i], max_steps=MAX_STEPS, sample_lib=sample_lib, popsize=POPSIZE, n_offspring=N_OFFSPRING, onset_frac=1, zeta=ZETA, population=pop, mutator=mutator, onsets=[0])
    results.append(result)

100%|██████████| 1000/1000 [00:14<00:00, 69.99it/s, Best individual: Fitness: 1.3987831078506097 | (Cello, tr171VCNVM, e3), (Trumpet, tr211TRNOM, c4), (Trumpet, tr211TRNOM, c6), (Piano, Alicias_Keys, e5), (Piano, The_Giant_soft, c8)]  


With a-priori knowledge, approximate only instruments

In [24]:
def get_valid_sample(sample_lib:SampleLibrary, pitch:Pitch):
    instrument, style = sample_lib.get_random_instrument_for_pitch(pitch=pitch)
    return sample_lib.get_sample(instrument=instrument, style=style, pitch=pitch)
# Initialize populations with a-priori knowledge
populations = [Population() for _ in range(len(target_chords))]
for i, pop in enumerate(populations):
    for j in range(POPSIZE):
        individual = BaseIndividual()
        for note in target_chords[i]:
            individual.samples.append(get_valid_sample(sample_lib, note[1]))
        individual.fitness = fitness(target_mixes[i], individual.to_mixdown())
        individual.fitness_per_onset.append(individual.fitness)
        pop.insert_individual(individual)
# Only allow the mutate_instrument mutation
mutator = Mutator(sample_library=sample_lib, alpha=ALPHA, beta=BETA, l_bound=L_BOUND, u_bound=U_BOUND, choose_mutation_p=[0, 1, 0]) 

In [25]:
results = []
for i, pop in enumerate(populations):
    result = approximate_piece(target_y=target_mixes[i], max_steps=MAX_STEPS, sample_lib=sample_lib, popsize=POPSIZE, n_offspring=N_OFFSPRING, onset_frac=1, zeta=ZETA, population=pop, mutator=mutator, onsets=[0])
    results.append(result)

100%|██████████| 1000/1000 [00:16<00:00, 60.29it/s, Best individual: Fitness: 0.2759647722942073 | (Cello, tr171VCNOM, c3), (Saxophone, Saxophones_Essential, c4), (Trumpet, trBbTpt, e4), (Flute, tr331FLNOM, g4), (Piano, tr013PFPEM, c5)]                                                


Without a-priori knowledge, approximate both instruments and pitches

In [ ]:
results = []
for i, pop in enumerate(populations):
    result = approximate_piece(target_y=target_mixes[i], max_steps=MAX_STEPS, sample_lib=sample_lib, popsize=POPSIZE, n_offspring=N_OFFSPRING, onset_frac=1, zeta=ZETA, onsets=[0])
    results.append(result)